In [ ]:
import polars as pl 
import matplotlib.pyplot as plt

In [ ]:
hojas = [
    "Aluminium",
    "Copper",
    "Lead",
    "Zinc",
    "Nickel",
    "Polyethylene", 
    "Polypropylene",
    "Cement",
    "Concrete",
    "Glass",
    "Wood"
]
Categorias = [
    "Acidification: terrestrial",
    "Climate change",
    "Ecotoxicity: freshwater",
    "Ecotoxicity: marine",
    "Ecotoxicity: terrestrial",
    "Energy resources: non-renewable, fossil",
    "Eutrophication: freshwater",
    "Eutrophication: marine",
    "Human toxicity: carcinogenic",
    "Human toxicity: non-carcinogenic",
    "Ionising radiation",
    "Land use",
    "Material resources: metals/minerals",
    "Ozone depletion",
    "Particulate matter formation",
    "Photochemical oxidant formation: human health",
    "Photochemical oxidant formation: terrestrial ecosystems",
    "Water use"
]

In [ ]:
# Ruta a tu Excel
ruta_excel = "Materiales.xlsx"

# Columnas de impacto que quieres graficar
impacto_x = Categorias[0]
impacto_y = Categorias[1]

# Leer todas las hojas en un diccionario de DataFrames
data = pl.read_excel(ruta_excel, sheet_name=hojas)

for hoja in hojas:
    print(f"\n--- Procesando hoja: {hoja} ---")
    df = pl.read_excel(ruta_excel, sheet_name=hoja)

    if impacto_x not in df.columns or impacto_y not in df.columns or "Material" not in df.columns:
        print(f"Saltando hoja {hoja}, faltan columnas necesarias")
        continue


    # Convertir columnas de impacto a numéricas (forzar errores a null)
    df = df.with_columns([
        pl.col(impacto_x).cast(pl.Float64, strict=False).alias(impacto_x),
        pl.col(impacto_y).cast(pl.Float64, strict=False).alias(impacto_y)
    ])
    
    # Quitar filas donde ambos impactos no son números
    df_num = df.filter(pl.col(impacto_x).is_not_null() & pl.col(impacto_y).is_not_null())


    # Resumen estadístico
    resumen = {}
    for col in [impacto_x, impacto_y]:
        maximo_val = df_num[col].max()
        minimo_val = df_num[col].min()

        maximo_mat = df_num.filter(pl.col(col) == maximo_val)["Material"].to_list()
        minimo_mat = df_num.filter(pl.col(col) == minimo_val)["Material"].to_list()
        ceros_mat = df_num.filter(pl.col(col) == 0)["Material"].to_list()
        negativos_mat = df_num.filter(pl.col(col) < 0)["Material"].to_list()

        resumen[col] = {
            "maximo": (maximo_val, maximo_mat),
            "minimo": (minimo_val, minimo_mat),
            "ceros": ceros_mat,
            "negativos": negativos_mat,
        }

    print("Resumen:", resumen)

   # Crear copia para graficar sin los máximos
    df_plot = df_num.filter(
        (pl.col(impacto_x) != resumen[impacto_x]["maximo"][0]) &
        (pl.col(impacto_y) != resumen[impacto_y]["maximo"][0])
    )

    # Convertir a listas para graficar
    x_vals = df_plot[impacto_x].to_list()
    y_vals = df_plot[impacto_y].to_list()

    # Graficar scatter
    plt.figure(figsize=(7, 5))
    plt.scatter(x_vals, y_vals, alpha=0.7, label="Materiales")
    plt.xlabel(impacto_x)
    plt.ylabel(impacto_y)
    plt.title(f"Scatter Plot - {hoja}")
    plt.legend()
    plt.grid(True, linestyle="--", alpha=0.5)
    plt.show()